In [ ]:
import pandas as pd
import json
csv_path = '../data/csv/data_2025_12_28-14h_07m.csv'
json_path = '../data/dataset_autos_V2_cleaned.jsonl'
df = pd.read_csv(csv_path)
df.sample(1)

#### Configuracion

In [ ]:
from IPython.display import display, HTML

# Configuración visual para Jupyter (Ancho completo)
display(HTML("<style>.container { width:100% !important; }</style>"))

# Muestra todas las columnas evitando el "..."
pd.set_option('display.max_columns', None)

# Ajustar el ancho de visualización 
pd.set_option('display.width', 1000)

# Muestra todo el contenido de la celda evitando que corte textos largos
pd.set_option('display.max_colwidth', None)

# Evita la notación científica (ej: 1.5e6 -> 1500000.00)
pd.set_option('display.float_format', '{:.2f}'.format)

### Estadisticas basicas

In [ ]:
df.sample(1)

print("---2. Dimensiones ---")
print(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}, Autos: {df.shape[0]/6}")
print("\n---3. Tipos de datos ---")
print(df.info())

print("\n---4. Conteo de autos unicos")
unique_cars = df['ID_Auto'].nunique()
print(f"Total de autos unicos: {unique_cars}")

print("---5. Estadisticas Rapidas ---")
print(df[['Precio', 'Mensualidad', 'Tasa', 'Enganche_Simulado', 'Enganche_Min', 'Enganche_Max']].describe())

### Encontrando Insights

In [ ]:
def get_car_data(id_car, df):
    return df[df['ID_Auto'] == id_car]

### Bono por Financiamiento y Precio de Contado (PoC).

El auto vale 400k, pero si te endeudas con nosotros, te lo dejamos en 380k, una diferencia de 20k. Sin embargo, no es real porque recuperan ese descuento (y mucho más) con los intereses que te cobran. 
En este pequeño ejemplo se haya una pequeña correlacion oculta, donde a mayor precio, mayor es el porcentaje de descuento: en un auto caro, el margen de ganancia es mayor y el préstamo es más grande (más intereses para Kavak), así que pueden ofrecer un "Bono" más agresivo para engancharte. Esto sugiere que el precio de contado no es aleatorio sino una variable dependiente del precio y la categoría del auto donde incluso con una regresión lineal simple puede utilizarse para imputarlo, es decir, inventarlo con base científica.

Auto caro (391k) -> Descuento ~6.5%
Auto medio (375k) -> Descuento ~6.2%
Auto "barato" (360k) -> Descuento ~4.6%

In [ ]:
def get_contado_porcentaje(precio_contado, precio_meses):
    return ((precio_contado - precio_meses)*100)/precio_meses

print(get_contado_porcentaje(399999,375999))
print(get_contado_porcentaje(391999,367999))
print(get_contado_porcentaje(385999,368999))
print(get_contado_porcentaje(375999,353999))
print(get_contado_porcentaje(360999,340999))

"""
Salida:
6.3829956994566475
6.521756852600143
4.607058555714243
6.214706821205709
5.865119839061111
"""

### Modelado del dataframe final limpio

In [ ]:
### Extraccion de Marca, Modelo, Anio, Version y Tipo del slug.
df_fin = pd.read_csv(csv_path)

#Diccionario {'auto_id':'Nombre del auto'}
autos_dict = {}
with open('../data/dataset_autos_V2.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        car = json.loads(line)
        name = car['slug'].split('/')[-1].replace('-', ' ').title()
        autos_dict[car['id']] = name

#VLOOKUP by ID
df_fin['Nombre'] = df_fin['ID_Auto'].astype(str).map(autos_dict)

# Separate name into ['Brand', 'Model', 'Version', 'Type', 'Year']
df_fin[['Brand', 'Model', 'Version', 'Type', 'Year']] = (
    df_fin['Nombre']
    .str.strip()
    .str.split(' ', expand=True)
)
df_fin = df_fin.drop(columns=['Nombre'])

# Calculo de datos financieros
df_fin['Total_a_Pagar'] = (df_fin['Mensualidad'] * df_fin['Plazo']) + df_fin['Enganche_Simulado'] # Plazo * Mensualidad + enganche 

df_fin['Interes'] = df_fin['Total_a_Pagar'] - df_fin['Precio'] # Costo extra
df_fin['Interes_%'] = (df_fin['Interes'] * 100) / df_fin['Precio'] # % de costo extra respecto al precio  

df_fin['Enganche_Min_%'] = (df['Enganche_Min'] * 100) / df_fin['Precio']
df_fin['Enganche_Max_%'] = (df['Enganche_Max'] * 100) / df_fin['Precio']

# Casting
df_fin['ID_Auto'] = df_fin['ID_Auto'].astype('string')
df_fin['Brand'] = df_fin['Brand'].astype('category')
df_fin['Model'] = df_fin['Model'].astype('string')
df_fin['Version'] = df_fin['Version'].astype('string')
df_fin['Year'] = df_fin['Year'].astype('Int64')
df_fin['Plazo'] = df_fin['Plazo'].astype('category')
df_fin['Type'] = df_fin['Type'].astype('category')

### Integracion de datos faltantes (km, sucursal, caja, oferta)
df_prod = pd.read_json('../data/dataset_autos_V2_cleaned.jsonl', lines=True)

df_prod['id'] = df_prod['id'].astype('string')
df_prod = df_prod.drop(columns=['slug', 'price', 'details']) # Elimina cols innecesarias
df_prod.rename(columns={'id':'ID_Auto', 'year':'Year', 'sucursal':'Sucursal', 'caja':'Caja', 'km':'Km', 'oferta':'Oferta'}, inplace=True)

df_merged = df_fin.merge(
    df_prod, 
    how='inner', 
    on='ID_Auto', 
    suffixes=('_fin', '_prod')
    )

if (df_merged[df_merged['Year_fin'] != df_merged['Year_prod']]).empty:
    #print("Los datos son identicos. Eliminando...")
    df_merged.rename(columns={'Year_prod':'Year', 'Type':'Tipo'}, inplace=True)
    df_merged = df_merged.drop(columns=['Year_fin'])
    

# Casting
df_merged['Sucursal'] = df_merged['Sucursal'].astype('category')
df_merged['Year'] = df_merged['Year'].astype('Int64')
df_merged['Caja'] = df_merged['Caja'].astype('category')
df_merged['Km'] = df_merged['Km'].astype('Int64')
df_merged['Oferta'] = df_merged['Oferta'].astype('category')

In [ ]:
df_merged.info()

In [ ]:
df_merged.to_csv('../data/csv/cleaned_final_csv.csv', encoding='utf-8', index=False)

### Matriz de Correlación (Heatmap)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

cols_interes = ['Precio', 'Year', 'Km', 'Tasa', 'Mensualidad', 'Enganche_Min_%']

df_subset = df_merged[cols_interes]

matriz_corr = df_subset.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(matriz_corr,
            annot=True,
            cmap='coolwarm',
            fmt=".2f")
plt.show()

### Regresion para evaluacion de precio justo

### Insights

In [ ]:
print("--- TOP 5 AUTOS CON LAS MENSUALIDADES MAS BAJA (Plazo 48) y mayor porcentaje de enganche min y max ---")
df_48 = df_fin[df_fin['Plazo'] == 48].copy()

best_rates = df_48.sort_values('Mensualidad').head(5)
print(
    best_rates[['Brand','Model', 'Year', 'Mensualidad', 'Precio', 'Interes_Total', 'Total_a_Pagar', 'Porcentaje_Extra', 
         'Enganche_Min_%', 'Enganche_Max_%']].sort_values('Enganche_Max_%', ascending=False))

In [ ]:
car_data  = get_car_data(458889, df_fin).sort_values('Plazo', ascending=True)
car_data.loc[:, ['Brand', 'Model', 'Year', 'Type']]

In [ ]:
# Smart Buys
# Autos donde el financiamiento sea menor al 40% del valor del auto con un plazo de 48 meses
smart_buys = df_fin[
    (df_fin['Plazo'] == 48) &
    #(df_fin['Porcentaje_Extra'] < 45) & # Limite porcentual maximo de financiamiento respecto al precio del auto
    (df_fin['Total_a_Pagar'] < 250000)
].sort_values('Porcentaje_Extra', ascending=True)

if not smart_buys.empty:
    print(smart_buys[['ID_Auto', 'Brand', 'Model', 'Year', 'Type', 'Plazo', 'Precio', 'Interes_Total', 'Mensualidad', 'Total_a_Pagar', 'Porcentaje_Extra']].head(10))
else:
    print("No hay autos con financiamiento en este porcentaje")

In [ ]:

worst_deals = df_fin[df_fin['Plazo'] == 60].sort_values('Porcentaje_Extra', ascending=False).head(10)
print("Los 10 autos donde pagas más intereses proporcionales (a 60 meses):")
print(worst_deals[['Brand', 'Model', 'Plazo', 'Precio', 'Total_a_Pagar', 'Porcentaje_Extra']])

In [ ]:
#The cashflow King
print("\n--- AUTOS CON VALOR MAYOR A 300k CON MENSUALIDAD MENORES A 10K ---")

luxury_cheap = df_fin[
    (df_fin['Precio'] > 300000) &
    (df_fin['Mensualidad'] < 10000)
].sort_values('Porcentaje_Extra', ascending=True).head()



print(luxury_cheap[['ID_Auto', 'Brand', 'Model', 'Type', 'Year', 'Precio', 'Tasa', 'Plazo', 'Mensualidad', 'Total_a_Pagar', 'Porcentaje_Extra']])

In [ ]:
# Filtramos específicamente ese Nissan Sentra usando tu DataFrame ya cargado
# (Asegúrate de tener df_fin cargado del paso anterior)

sentra_id = 1003531 # Lo vi en tu tabla anterior del análisis "Premium"
sentra_real = df_fin[
    (df_fin['ID_Auto'] == sentra_id) & 
    (df_fin['Plazo'] == 72)
].copy()

# Calculamos la realidad
sentra_real['Total_a_Pagar'] = (sentra_real['Mensualidad'] * sentra_real['Plazo']) + sentra_real['Enganche_Simulado']
sentra_real['Interes_Pagado'] = sentra_real['Total_a_Pagar'] - sentra_real['Precio']

print("--- 💸 LA VERDAD DEL NISSAN SENTRA 2023 ---")
print(f"Precio de Contado:      ${sentra_real['Precio'].values[0]:,.2f}")
print(f"Enganche Mínimo (16%):  ${sentra_real['Enganche_Simulado'].values[0]:,.2f}")
print(f"Tu Mensualidad Real:    ${sentra_real['Mensualidad'].values[0]:,.2f}")
print(f"Total a pagar (6 años): ${sentra_real['Total_a_Pagar'].values[0]:,.2f}")
print(f"Intereses Extra:        ${sentra_real['Interes_Pagado'].values[0]:,.2f}")
print(f"Intere Porcentual Extra: {sentra_real['Porcentaje_Extra'].values[0]:,.2f}%")